# Capstone Project - The Battle of the Neighborhoods (Week 2)

## Introduction: Business Problem

In this project we will try to help ABC Company Ltd, a traditional Jewish chained restaurant management group with mid to high price range, to find the best location in Hong Kong to open their first flagship restaurant in Asia. 

As they provide mid to high range food and beverages, we will find the location based on: 
- the income of the population in the area
- the age range of the population in the area (age 20-64)
- the rental cost of the area 
- the types of restaurant nearby

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data

Based on definition of our problem, factors that will influence our decission are:

- income
- age 
- rent 
- types of restaurant in the same area 


Following data sources will be needed to extract/generate the required information:

- Demographics information from Wikipedia
- Demographics information and rental cost from Esri China
- Number of restaurants and their type and location in every neighborhood will be obtained using Foursquare API

## Demographics information from Wikipedia

Let's find out what area the districts in Hong Kong and the basic demographics.

In [1]:
import pandas as pd
import requests
wiki_population = requests.get('https://en.wikipedia.org/wiki/Districts_of_Hong_Kong').text

In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(wiki_population,'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Districts of Hong Kong - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XeO7mApAIDAAALgkYpAAAAAY","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Districts_of_Hong_Kong","wgTitle":"Districts of Hong Kong","wgCurRevisionId":928758203,"wgRevisionId":928758203,"wgArticleId":151994,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Webarchive template wayback links","Engv

In [3]:
table = soup.find('table',{'class':'wikitable sortable'})
table

<table class="wikitable sortable">
<tbody><tr>
<th>District
</th>
<th>Chinese
</th>
<th>Population<sup class="noprint Inline-Template" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Manual_of_Style/Dates_and_numbers#Chronological_items" title="Wikipedia:Manual of Style/Dates and numbers"><span title="The time period mentioned near this tag is ambiguous. (December 2019)">when?</span></a></i>]</sup> <sup class="reference" id="cite_ref-6"><a href="#cite_note-6">[6]</a></sup>
</th>
<th>Area<br/>(km²)
</th>
<th>Density<br/>(/km²)
</th>
<th>Region
</th></tr>
<tr>
<td><a href="/wiki/Central_and_Western_District" title="Central and Western District"><span class="nowrap">Central and Western</span></a></td>
<td><span lang="zh-HK" title="Chinese language text">中西區</span></td>
<td align="right">244,600</td>
<td align="right">12.44</td>
<td align="right">19,983.92</td>
<td><span class="nowrap"><a href="/wiki/Hong_Kong_Island" title="Hong Kong Island">Hong Kong Island</a></span>
</td></tr>

In [4]:
print(table.tr.text)


District

Chinese

Population[when?] [6]

Area(km²)

Density(/km²)

Region



In [5]:
headers="District, Chinese, Population, Area, Density, Region" 

In [6]:
l = []
for tr in table.find_all('tr'):
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
pd.DataFrame(l, columns=["District", "Chinese", "Population", "Area", "Density", "Region" ])

,District,Chinese,Population,Area,Density,Region
0,None,None,None,None,None,None
1,Central and Western,中西區,"244,600",12.44,"19,983.92",Hong Kong Island\n
2,Eastern,東區,"574,500",18.56,"31,217.67",Hong Kong Island\n
3,Southern,南區,"269,200",38.85,"6,962.68",Hong Kong Island\n
4,Wan Chai,灣仔區,"150,900",9.83,"15,300.10",Hong Kong Island\n
5,Sham Shui Po,深水埗區,"390,600",9.35,"41,529.41",Kowloon\n
6,Kowloon City,九龍城區,"405,400",10.02,"40,194.70",Kowloon\n
7,Kwun Tong,觀塘區,"641,100",11.27,"56,779.05",Kowloon\n
8,Wong Tai Sin,黃大仙區,"426,200",9.30,"45,645.16",Kowloon\n
9,Yau Tsim Mong,油尖旺區,"318,100",6.99,"44,864.09",Kowloon\n


## Create dataframe with income distribution

Let's get the Working Population Distribution by Income Distribution from: http://opendata.esrichina.hk/datasets/working-population-distribution-by-income-distribution-in-2016/data?geometry=113.150%2C22.135%2C115.127%2C22.580

In [7]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_5890fd62c0bd4f4c9823b10bfd67e1ab = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Ci5Diz2FT2gFgwffcOxMs0kN-iihhb0Gp9FwGBPGkRXF',
    ibm_auth_endpoint="https://iam.au-syd.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.ap-geo.objectstorage.service.networklayer.com')

body = client_5890fd62c0bd4f4c9823b10bfd67e1ab.get_object(Bucket='thebattleofneighborhoods-donotdelete-pr-m6fr5gdubrolvp',Key='Working_Population_Distribution_by_Income_Distribution.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

income_data = pd.read_csv(body)
income_data.head()


,English_Name,income_less_than_6000,F6000_to_less_than_10000,F10000_to_less_than_20000,F20000_to_less_than_30000,equal_or_more_than30000,Total
0,WONG TAI SIN,23511,27046,99256,32044,27420,209277
1,KWAI TSING,30404,36719,122595,37173,32536,259427
2,SHAM SHUI PO,28509,29595,83472,27203,36786,205565
3,YAU TSIM MONG,27612,23779,63191,25263,43239,183084
4,KOWLOON CITY,38036,21940,74167,31089,51992,217224


According to the Hong Kong Government data at: https://www.censtatd.gov.hk/hkstat/sub/so210.jsp , the median monthly wage of all industry sectors is hkd 17,500. Since our clients only targets at higher income groups, therefore, we will only look at the income groups above hkd 30,000.

In [8]:
income_data.sort_values(by='equal_or_more_than30000', ascending=False)

,English_Name,income_less_than_6000,F6000_to_less_than_10000,F10000_to_less_than_20000,F20000_to_less_than_30000,equal_or_more_than30000,Total
8,SHA TIN,48250,29416,125433,56009,76094,335202
12,EASTERN,49090,33771,97256,43268,72988,296373
6,SAI KUNG,37544,20596,85603,41979,59610,245332
4,KOWLOON CITY,38036,21940,74167,31089,51992,217224
10,CENTRAL & WESTERN,32397,11837,31911,16438,46951,139534
14,YUEN LONG,40362,31981,133786,49125,46422,301676
5,KWUN TONG,36385,42588,146857,45026,44721,315577
3,YAU TSIM MONG,27612,23779,63191,25263,43239,183084
15,TSUEN WAN,23193,17259,60360,28195,39779,168786
13,TUEN MUN,29221,27529,111462,40105,37017,245334


# Create dataframe with spatial distribution of age 

Let's get the Age Distribution from: http://opendata.esrichina.hk/datasets/population-distribution-by-quinquennial-age-group-2016-1

In [9]:

body = client_5890fd62c0bd4f4c9823b10bfd67e1ab.get_object(Bucket='thebattleofneighborhoods-donotdelete-pr-m6fr5gdubrolvp',Key='Population_distribution_by_age_group.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

age_data = pd.read_csv(body)
age_data.head()


,English_Name,Age_0_to_4,Age_5_to_9,Age_10_to_14,Age_15_to_19,Age_20_to_24,Age_25_to_29,Age_30_to_34,Age_35_to_39,Age_40_to_44,Age_45_to_49,Age_50_to_54,Age_55_to_59,Age_60_to_64,Age_65_to_69,Age_70_to_74,Age_Over_75
0,WONG TAI SIN,14529,16767,14841,20313,27154,28101,30654,31169,31429,32337,37869,37294,29524,23493,13189,36572
1,KWAI TSING,19220,19693,19388,25446,31313,36107,40875,40584,39885,38781,44190,43395,34908,28234,18508,40045
2,SHAM SHUI PO,17104,16703,13808,18432,24403,28828,33246,31951,31888,31597,34472,33171,25793,20804,11585,32084
3,YAU TSIM MONG,13063,12873,11153,15500,22642,26533,27191,25855,25949,25909,31943,29548,23161,18707,9162,23781
4,KOWLOON CITY,17189,17609,13802,18367,26325,31147,34215,31973,30705,30208,37240,36832,28972,22785,11245,30118


 Since our client would like to target the working classes, i.e. around age 25 - 40, we will sort it by these age groups and see which areas has the most concentrated distribution with age 25-40. 

Age 25-29: 

In [10]:
age_data.sort_values(by='Age_25_to_29', ascending=False)

,English_Name,Age_0_to_4,Age_5_to_9,Age_10_to_14,Age_15_to_19,Age_20_to_24,Age_25_to_29,Age_30_to_34,Age_35_to_39,Age_40_to_44,Age_45_to_49,Age_50_to_54,Age_55_to_59,Age_60_to_64,Age_65_to_69,Age_70_to_74,Age_Over_75
5,KWUN TONG,24651,25790,24085,29994,39275,44079,50596,49574,48040,47895,54418,54456,44429,34925,20972,55362
8,SHA TIN,23891,26735,24172,32394,40053,42780,48413,48220,52313,57622,59368,55011,43063,35609,19155,50455
14,YUEN LONG,25063,26144,22514,30269,40147,42190,49539,50652,48669,45187,50746,49067,41257,33371,19336,40027
12,EASTERN,21376,21341,18177,24592,31461,37787,44558,45014,42921,42350,49580,46954,36609,32032,16250,44032
1,KWAI TSING,19220,19693,19388,25446,31313,36107,40875,40584,39885,38781,44190,43395,34908,28234,18508,40045
6,SAI KUNG,18809,18344,15610,21277,28195,32872,38610,37362,36012,36183,40366,39061,31297,25631,13744,28491
13,TUEN MUN,17468,19069,18003,23930,30365,32644,36222,37463,39535,40482,43381,42940,35508,26162,14754,31373
4,KOWLOON CITY,17189,17609,13802,18367,26325,31147,34215,31973,30705,30208,37240,36832,28972,22785,11245,30118
2,SHAM SHUI PO,17104,16703,13808,18432,24403,28828,33246,31951,31888,31597,34472,33171,25793,20804,11585,32084
0,WONG TAI SIN,14529,16767,14841,20313,27154,28101,30654,31169,31429,32337,37869,37294,29524,23493,13189,36572


Age 30 - 34: 

In [11]:
age_data.sort_values(by='Age_30_to_34', ascending=False)

,English_Name,Age_0_to_4,Age_5_to_9,Age_10_to_14,Age_15_to_19,Age_20_to_24,Age_25_to_29,Age_30_to_34,Age_35_to_39,Age_40_to_44,Age_45_to_49,Age_50_to_54,Age_55_to_59,Age_60_to_64,Age_65_to_69,Age_70_to_74,Age_Over_75
5,KWUN TONG,24651,25790,24085,29994,39275,44079,50596,49574,48040,47895,54418,54456,44429,34925,20972,55362
14,YUEN LONG,25063,26144,22514,30269,40147,42190,49539,50652,48669,45187,50746,49067,41257,33371,19336,40027
8,SHA TIN,23891,26735,24172,32394,40053,42780,48413,48220,52313,57622,59368,55011,43063,35609,19155,50455
12,EASTERN,21376,21341,18177,24592,31461,37787,44558,45014,42921,42350,49580,46954,36609,32032,16250,44032
1,KWAI TSING,19220,19693,19388,25446,31313,36107,40875,40584,39885,38781,44190,43395,34908,28234,18508,40045
6,SAI KUNG,18809,18344,15610,21277,28195,32872,38610,37362,36012,36183,40366,39061,31297,25631,13744,28491
13,TUEN MUN,17468,19069,18003,23930,30365,32644,36222,37463,39535,40482,43381,42940,35508,26162,14754,31373
4,KOWLOON CITY,17189,17609,13802,18367,26325,31147,34215,31973,30705,30208,37240,36832,28972,22785,11245,30118
2,SHAM SHUI PO,17104,16703,13808,18432,24403,28828,33246,31951,31888,31597,34472,33171,25793,20804,11585,32084
0,WONG TAI SIN,14529,16767,14841,20313,27154,28101,30654,31169,31429,32337,37869,37294,29524,23493,13189,36572


Age 35-39: 

In [12]:
age_data.sort_values(by='Age_35_to_39', ascending=False)

,English_Name,Age_0_to_4,Age_5_to_9,Age_10_to_14,Age_15_to_19,Age_20_to_24,Age_25_to_29,Age_30_to_34,Age_35_to_39,Age_40_to_44,Age_45_to_49,Age_50_to_54,Age_55_to_59,Age_60_to_64,Age_65_to_69,Age_70_to_74,Age_Over_75
14,YUEN LONG,25063,26144,22514,30269,40147,42190,49539,50652,48669,45187,50746,49067,41257,33371,19336,40027
5,KWUN TONG,24651,25790,24085,29994,39275,44079,50596,49574,48040,47895,54418,54456,44429,34925,20972,55362
8,SHA TIN,23891,26735,24172,32394,40053,42780,48413,48220,52313,57622,59368,55011,43063,35609,19155,50455
12,EASTERN,21376,21341,18177,24592,31461,37787,44558,45014,42921,42350,49580,46954,36609,32032,16250,44032
1,KWAI TSING,19220,19693,19388,25446,31313,36107,40875,40584,39885,38781,44190,43395,34908,28234,18508,40045
13,TUEN MUN,17468,19069,18003,23930,30365,32644,36222,37463,39535,40482,43381,42940,35508,26162,14754,31373
6,SAI KUNG,18809,18344,15610,21277,28195,32872,38610,37362,36012,36183,40366,39061,31297,25631,13744,28491
4,KOWLOON CITY,17189,17609,13802,18367,26325,31147,34215,31973,30705,30208,37240,36832,28972,22785,11245,30118
2,SHAM SHUI PO,17104,16703,13808,18432,24403,28828,33246,31951,31888,31597,34472,33171,25793,20804,11585,32084
0,WONG TAI SIN,14529,16767,14841,20313,27154,28101,30654,31169,31429,32337,37869,37294,29524,23493,13189,36572


# Create dataframe with shop rental price in different areas 

Let's get the Shop Rent Valuation from: http://opendata.esrichina.hk/datasets/summary-statistics-on-valuation-list-and-government-rent-roll-of-hong-kong/data

In [13]:

body = client_5890fd62c0bd4f4c9823b10bfd67e1ab.get_object(Bucket='thebattleofneighborhoods-donotdelete-pr-m6fr5gdubrolvp',Key='Summary_Statistics_on_Shop_rent_Valuation.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

rent_data = pd.read_csv(body)
rent_data.head()


,DISTRICT,Shop_Number,鋪位數量,Shop_Rateable_Value_000
0,WONG TAI SIN,3427,3427,2092898
1,KWAI TSING,3888,3888,2496396
2,SHAM SHUI PO,9430,9430,4403466
3,YAU TSIM MONG,21099,21099,21964082
4,KOWLOON CITY,7728,7728,3252365


With the concern of cost, we would like to see the rent from low to high in different areas: 

In [14]:
rent_data.sort_values(by='Shop_Rateable_Value_000')

,DISTRICT,Shop_Number,鋪位數量,Shop_Rateable_Value_000
17,SOUTHERN,2263,2263,1318179
7,TAI PO,2633,2633,1561385
0,WONG TAI SIN,3427,3427,2092898
9,NORTH,2870,2870,2143855
1,KWAI TSING,3888,3888,2496396
6,SAI KUNG,3160,3160,2501343
13,TUEN MUN,5632,5632,3124248
4,KOWLOON CITY,7728,7728,3252365
15,TSUEN WAN,5528,5528,3494721
14,YUEN LONG,7559,7559,3843112


# Scoring System

#### Based on the income, age, rent we find above, we can build a scoring system to get a standardized score based the category being divided by its median score and then multiplied by a factor of given importance. 

First, we will find the median of different items, including: income, age(3 groups), and rent. According to our client's will, we will prioritize income and age, then rent will be the less important factor. 
The weight of different factors will be: 
- Income = 0.4
- Age = 0.5 (Age 25-29 = 0.15; Age 30-34 = 0.2 ; Age 35-39 = 0.15)
- Rent = 0.1

### Score of income

Median of income was provided based on government statistics: hkd 17,500

In [15]:
income_data['income_score'] = (income_data['equal_or_more_than30000']/17500)*0.4
income_data.head(5)

,English_Name,income_less_than_6000,F6000_to_less_than_10000,F10000_to_less_than_20000,F20000_to_less_than_30000,equal_or_more_than30000,Total,income_score
0,WONG TAI SIN,23511,27046,99256,32044,27420,209277,0.626743
1,KWAI TSING,30404,36719,122595,37173,32536,259427,0.743680
2,SHAM SHUI PO,28509,29595,83472,27203,36786,205565,0.840823
3,YAU TSIM MONG,27612,23779,63191,25263,43239,183084,0.988320
4,KOWLOON CITY,38036,21940,74167,31089,51992,217224,1.188389


In [16]:
income_data.sort_values(by='income_score', ascending=False)

,English_Name,income_less_than_6000,F6000_to_less_than_10000,F10000_to_less_than_20000,F20000_to_less_than_30000,equal_or_more_than30000,Total,income_score
8,SHA TIN,48250,29416,125433,56009,76094,335202,1.739291
12,EASTERN,49090,33771,97256,43268,72988,296373,1.668297
6,SAI KUNG,37544,20596,85603,41979,59610,245332,1.362514
4,KOWLOON CITY,38036,21940,74167,31089,51992,217224,1.188389
10,CENTRAL & WESTERN,32397,11837,31911,16438,46951,139534,1.073166
14,YUEN LONG,40362,31981,133786,49125,46422,301676,1.061074
5,KWUN TONG,36385,42588,146857,45026,44721,315577,1.022194
3,YAU TSIM MONG,27612,23779,63191,25263,43239,183084,0.988320
15,TSUEN WAN,23193,17259,60360,28195,39779,168786,0.909234
13,TUEN MUN,29221,27529,111462,40105,37017,245334,0.846103


### Score of Age 

Let's get the median of the 3 age groups that we target: 

In [17]:
import pandas as pd
import numpy as np
print('Median of age 25 to 29 is', age_data['Age_25_to_29'].median())
print('Median of age 30 to 34 is', age_data['Age_30_to_34'].median())
print('Median of age 35 to 39 is', age_data['Age_35_to_39'].median())

Median of age 25 to 29 is 28464.5
Median of age 30 to 34 is 31950.0
Median of age 35 to 39 is 31560.0


Therefore, let's make use of the median to calculate the scores of each age group.

In [18]:
age_data['score_Age_25_to_29'] = (age_data['Age_25_to_29']/28464.5)*0.15
age_data['score_Age_30_to_34'] = (age_data['Age_25_to_29']/31950)*0.2
age_data['score_Age_35_to_39'] = (age_data['Age_25_to_29']/31560)*0.15

age_data.head(5)

,English_Name,Age_0_to_4,Age_5_to_9,Age_10_to_14,Age_15_to_19,Age_20_to_24,Age_25_to_29,Age_30_to_34,Age_35_to_39,Age_40_to_44,Age_45_to_49,Age_50_to_54,Age_55_to_59,Age_60_to_64,Age_65_to_69,Age_70_to_74,Age_Over_75,score_Age_25_to_29,score_Age_30_to_34,score_Age_35_to_39
0,WONG TAI SIN,14529,16767,14841,20313,27154,28101,30654,31169,31429,32337,37869,37294,29524,23493,13189,36572,0.148084,0.175906,0.133560
1,KWAI TSING,19220,19693,19388,25446,31313,36107,40875,40584,39885,38781,44190,43395,34908,28234,18508,40045,0.190274,0.226022,0.171611
2,SHAM SHUI PO,17104,16703,13808,18432,24403,28828,33246,31951,31888,31597,34472,33171,25793,20804,11585,32084,0.151916,0.180457,0.137015
3,YAU TSIM MONG,13063,12873,11153,15500,22642,26533,27191,25855,25949,25909,31943,29548,23161,18707,9162,23781,0.139822,0.166091,0.126107
4,KOWLOON CITY,17189,17609,13802,18367,26325,31147,34215,31973,30705,30208,37240,36832,28972,22785,11245,30118,0.164136,0.194973,0.148037


Then, let's get the total score and see which areas have the highest score.

In [19]:
age_data['total_score'] = age_data['score_Age_25_to_29'] + age_data['score_Age_30_to_34']+ age_data['score_Age_35_to_39']
age_data.head()

,English_Name,Age_0_to_4,Age_5_to_9,Age_10_to_14,Age_15_to_19,Age_20_to_24,Age_25_to_29,Age_30_to_34,Age_35_to_39,Age_40_to_44,...,Age_50_to_54,Age_55_to_59,Age_60_to_64,Age_65_to_69,Age_70_to_74,Age_Over_75,score_Age_25_to_29,score_Age_30_to_34,score_Age_35_to_39,total_score
0,WONG TAI SIN,14529,16767,14841,20313,27154,28101,30654,31169,31429,...,37869,37294,29524,23493,13189,36572,0.148084,0.175906,0.133560,0.457550
1,KWAI TSING,19220,19693,19388,25446,31313,36107,40875,40584,39885,...,44190,43395,34908,28234,18508,40045,0.190274,0.226022,0.171611,0.587907
2,SHAM SHUI PO,17104,16703,13808,18432,24403,28828,33246,31951,31888,...,34472,33171,25793,20804,11585,32084,0.151916,0.180457,0.137015,0.469388
3,YAU TSIM MONG,13063,12873,11153,15500,22642,26533,27191,25855,25949,...,31943,29548,23161,18707,9162,23781,0.139822,0.166091,0.126107,0.432020
4,KOWLOON CITY,17189,17609,13802,18367,26325,31147,34215,31973,30705,...,37240,36832,28972,22785,11245,30118,0.164136,0.194973,0.148037,0.507146


In [20]:
age_data.sort_values(by='total_score', ascending=False)

,English_Name,Age_0_to_4,Age_5_to_9,Age_10_to_14,Age_15_to_19,Age_20_to_24,Age_25_to_29,Age_30_to_34,Age_35_to_39,Age_40_to_44,...,Age_50_to_54,Age_55_to_59,Age_60_to_64,Age_65_to_69,Age_70_to_74,Age_Over_75,score_Age_25_to_29,score_Age_30_to_34,score_Age_35_to_39,total_score
5,KWUN TONG,24651,25790,24085,29994,39275,44079,50596,49574,48040,...,54418,54456,44429,34925,20972,55362,0.232284,0.275925,0.209501,0.717710
8,SHA TIN,23891,26735,24172,32394,40053,42780,48413,48220,52313,...,59368,55011,43063,35609,19155,50455,0.225439,0.267793,0.203327,0.696559
14,YUEN LONG,25063,26144,22514,30269,40147,42190,49539,50652,48669,...,50746,49067,41257,33371,19336,40027,0.222330,0.264100,0.200523,0.686953
12,EASTERN,21376,21341,18177,24592,31461,37787,44558,45014,42921,...,49580,46954,36609,32032,16250,44032,0.199127,0.236538,0.179596,0.615261
1,KWAI TSING,19220,19693,19388,25446,31313,36107,40875,40584,39885,...,44190,43395,34908,28234,18508,40045,0.190274,0.226022,0.171611,0.587907
6,SAI KUNG,18809,18344,15610,21277,28195,32872,38610,37362,36012,...,40366,39061,31297,25631,13744,28491,0.173226,0.205772,0.156236,0.535234
13,TUEN MUN,17468,19069,18003,23930,30365,32644,36222,37463,39535,...,43381,42940,35508,26162,14754,31373,0.172025,0.204344,0.155152,0.531521
4,KOWLOON CITY,17189,17609,13802,18367,26325,31147,34215,31973,30705,...,37240,36832,28972,22785,11245,30118,0.164136,0.194973,0.148037,0.507146
2,SHAM SHUI PO,17104,16703,13808,18432,24403,28828,33246,31951,31888,...,34472,33171,25793,20804,11585,32084,0.151916,0.180457,0.137015,0.469388
0,WONG TAI SIN,14529,16767,14841,20313,27154,28101,30654,31169,31429,...,37869,37294,29524,23493,13189,36572,0.148084,0.175906,0.133560,0.457550


### Score of rent

Let's get the median of the shop rent: 

In [21]:
print('Median of shop rent is', rent_data['Shop_Rateable_Value_000'].median())

Median of shop rent is 3668916.5


Now will will find the score of shop rent based on the median and weight:

In [22]:
rent_data['rent_score'] = (rent_data['Shop_Rateable_Value_000']/3668916.5)*0.1
rent_data.head()

,DISTRICT,Shop_Number,鋪位數量,Shop_Rateable_Value_000,rent_score
0,WONG TAI SIN,3427,3427,2092898,0.057044
1,KWAI TSING,3888,3888,2496396,0.068042
2,SHAM SHUI PO,9430,9430,4403466,0.120021
3,YAU TSIM MONG,21099,21099,21964082,0.598653
4,KOWLOON CITY,7728,7728,3252365,0.088646


### Final Score

Great! Now we have all 3 scores of income, age, and rent. Let's add them up and see which areas has the highest total score. After that, we can focus on those areas and see whether there is any competitor nearby with locational data. 

In [23]:
left=income_data
right=age_data
table_1= pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)
table_1

,English_Name,income_less_than_6000,F6000_to_less_than_10000,F10000_to_less_than_20000,F20000_to_less_than_30000,equal_or_more_than30000,Total,income_score,Age_0_to_4,Age_5_to_9,...,Age_50_to_54,Age_55_to_59,Age_60_to_64,Age_65_to_69,Age_70_to_74,Age_Over_75,score_Age_25_to_29,score_Age_30_to_34,score_Age_35_to_39,total_score
0,CENTRAL & WESTERN,32397,11837,31911,16438,46951,139534,1.073166,7535,8494,...,22792,21175,15612,13613,6864,18821,0.097727,0.116088,0.088142,0.301956
1,EASTERN,49090,33771,97256,43268,72988,296373,1.668297,21376,21341,...,49580,46954,36609,32032,16250,44032,0.199127,0.236538,0.179596,0.615261
2,ISLANDS,12362,7792,29172,11942,18547,79815,0.423931,6948,6141,...,12606,13208,10718,8732,5078,10689,0.061988,0.073634,0.055908,0.191529
3,KOWLOON CITY,38036,21940,74167,31089,51992,217224,1.188389,17189,17609,...,37240,36832,28972,22785,11245,30118,0.164136,0.194973,0.148037,0.507146
4,KWAI TSING,30404,36719,122595,37173,32536,259427,0.743680,19220,19693,...,44190,43395,34908,28234,18508,40045,0.190274,0.226022,0.171611,0.587907
5,KWUN TONG,36385,42588,146857,45026,44721,315577,1.022194,24651,25790,...,54418,54456,44429,34925,20972,55362,0.232284,0.275925,0.209501,0.717710
6,NORTH,17908,17686,67822,24633,22537,150586,0.515131,13060,14177,...,26971,25573,20645,16473,9303,23498,0.109890,0.130535,0.099111,0.339536
7,SAI KUNG,37544,20596,85603,41979,59610,245332,1.362514,18809,18344,...,40366,39061,31297,25631,13744,28491,0.173226,0.205772,0.156236,0.535234
8,SHA TIN,48250,29416,125433,56009,76094,335202,1.739291,23891,26735,...,59368,55011,43063,35609,19155,50455,0.225439,0.267793,0.203327,0.696559
9,SHAM SHUI PO,28509,29595,83472,27203,36786,205565,0.840823,17104,16703,...,34472,33171,25793,20804,11585,32084,0.151916,0.180457,0.137015,0.469388


In [24]:
rent_data.rename(columns={'DISTRICT':'English_Name'},
                            inplace=True)
rent_data.head()

,English_Name,Shop_Number,鋪位數量,Shop_Rateable_Value_000,rent_score
0,WONG TAI SIN,3427,3427,2092898,0.057044
1,KWAI TSING,3888,3888,2496396,0.068042
2,SHAM SHUI PO,9430,9430,4403466,0.120021
3,YAU TSIM MONG,21099,21099,21964082,0.598653
4,KOWLOON CITY,7728,7728,3252365,0.088646


In [25]:
left=table_1
right=rent_data
Final_score_draft= pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)
Final_score_draft

,English_Name,income_less_than_6000,F6000_to_less_than_10000,F10000_to_less_than_20000,F20000_to_less_than_30000,equal_or_more_than30000,Total,income_score,Age_0_to_4,Age_5_to_9,...,Age_70_to_74,Age_Over_75,score_Age_25_to_29,score_Age_30_to_34,score_Age_35_to_39,total_score,Shop_Number,鋪位數量,Shop_Rateable_Value_000,rent_score
0,CENTRAL & WESTERN,32397,11837,31911,16438,46951,139534,1.073166,7535,8494,...,6864,18821,0.097727,0.116088,0.088142,0.301956,9138,9138,10319382,0.281265
1,EASTERN,49090,33771,97256,43268,72988,296373,1.668297,21376,21341,...,16250,44032,0.199127,0.236538,0.179596,0.615261,8345,8345,3928122,0.107065
2,ISLANDS,12362,7792,29172,11942,18547,79815,0.423931,6948,6141,...,5078,10689,0.061988,0.073634,0.055908,0.191529,2682,2682,5881483,0.160306
3,KOWLOON CITY,38036,21940,74167,31089,51992,217224,1.188389,17189,17609,...,11245,30118,0.164136,0.194973,0.148037,0.507146,7728,7728,3252365,0.088646
4,KWAI TSING,30404,36719,122595,37173,32536,259427,0.743680,19220,19693,...,18508,40045,0.190274,0.226022,0.171611,0.587907,3888,3888,2496396,0.068042
5,KWUN TONG,36385,42588,146857,45026,44721,315577,1.022194,24651,25790,...,20972,55362,0.232284,0.275925,0.209501,0.717710,5910,5910,4920459,0.134112
6,NORTH,17908,17686,67822,24633,22537,150586,0.515131,13060,14177,...,9303,23498,0.109890,0.130535,0.099111,0.339536,2870,2870,2143855,0.058433
7,SAI KUNG,37544,20596,85603,41979,59610,245332,1.362514,18809,18344,...,13744,28491,0.173226,0.205772,0.156236,0.535234,3160,3160,2501343,0.068177
8,SHA TIN,48250,29416,125433,56009,76094,335202,1.739291,23891,26735,...,19155,50455,0.225439,0.267793,0.203327,0.696559,4722,4722,5233384,0.142641
9,SHAM SHUI PO,28509,29595,83472,27203,36786,205565,0.840823,17104,16703,...,11585,32084,0.151916,0.180457,0.137015,0.469388,9430,9430,4403466,0.120021


Let's clean up the data and get the final score of each table

In [26]:
Final_score = Final_score_draft[['English_Name','income_score', 'total_score', 'rent_score']]
Final_score.head()

,English_Name,income_score,total_score,rent_score
0,CENTRAL & WESTERN,1.073166,0.301956,0.281265
1,EASTERN,1.668297,0.615261,0.107065
2,ISLANDS,0.423931,0.191529,0.160306
3,KOWLOON CITY,1.188389,0.507146,0.088646
4,KWAI TSING,0.743680,0.587907,0.068042


Let's add the scores to get a final score of each district, then find the top 5 district with highest score.

In [27]:
Final_score['final_score']= Final_score.iloc[:, -4:-1].sum(axis=1)
Final_score.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,English_Name,income_score,total_score,rent_score,final_score
0,CENTRAL & WESTERN,1.073166,0.301956,0.281265,1.375122
1,EASTERN,1.668297,0.615261,0.107065,2.283558
2,ISLANDS,0.423931,0.191529,0.160306,0.615461
3,KOWLOON CITY,1.188389,0.507146,0.088646,1.695535
4,KWAI TSING,0.743680,0.587907,0.068042,1.331587


#### Now we can find the top 5 districts with highest score based on our scoring system.

In [28]:
Final_score.sort_values(by='final_score', ascending=False)

,English_Name,income_score,total_score,rent_score,final_score
8,SHA TIN,1.739291,0.696559,0.142641,2.435851
1,EASTERN,1.668297,0.615261,0.107065,2.283558
7,SAI KUNG,1.362514,0.535234,0.068177,1.897748
17,YUEN LONG,1.061074,0.686953,0.104748,1.748027
5,KWUN TONG,1.022194,0.717710,0.134112,1.739904
3,KOWLOON CITY,1.188389,0.507146,0.088646,1.695535
16,YAU TSIM MONG,0.988320,0.432020,0.598653,1.420340
13,TUEN MUN,0.846103,0.531521,0.085155,1.377624
0,CENTRAL & WESTERN,1.073166,0.301956,0.281265,1.375122
4,KWAI TSING,0.743680,0.587907,0.068042,1.331587


#### Therefore, now we can see the top 5 districts are: 
- SHA TIN	
- EASTERN
- SAI KUNG
- YUEN LONG	
- KWUN TONG


As a result, we will be focusing on these 3 districts and deepdive into them to see the nearby restaurants in order to choose the best area to open the first shop for our client. 

# Foursquare API 


load library

In [29]:
import requests
!conda install -c conda-forge folium=0.5.0 --yes 
import folium

import numpy as np
import pandas as pd

import matplotlib.cm as cm
import matplotlib.colors as colors

from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-3.3.0               |           py36_0         747 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be 

I have created a csv file manually with the coordinates of the top 5 districts. 

In [31]:

body = client_5890fd62c0bd4f4c9823b10bfd67e1ab.get_object(Bucket='thebattleofneighborhoods-donotdelete-pr-m6fr5gdubrolvp',Key='district.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_hk = pd.read_csv(body)
df_hk.head()


,District,Latitude,Longitude
0,Sha Tin,22.377100,114.197400
1,Eastern,22.288500,114.192800
2,Sai Kung,22.381400,114.273330
3,Yuen Long,22.444500,114.022200
4,Kwun Tong,22.311774,114.226209


Now we will plot the districts on the map of Hong Kong by using Folium. 

In [33]:
latitude = 22.2793278
longitude = 114.1828131

map_hk = folium.Map(location=[latitude, longitude], zoom_start=12)


for lat, lng, label in zip(df_hk['Latitude'], df_hk['Longitude'], df_hk['District']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_hk)  
    
map_hk

### Let's use Foursquare API to collect information on other venues/competitors in the neighborhoods of Hong Kong

We're interested in venues in 'food' category, but only those that are proper restaurants - coffe shops, pizza places, bakeries etc. are not direct competitors so we don't care about those. So we will include in out list only venues that have 'restaurant' in category name. After that, we will also look into the "Jewish/ Israeli" restaurants to see if there is any direct competitor in the same area. 



In [34]:
CLIENT_ID = '5LSHG4JHERQ4SHO4D4RFRC50EXWRAXRX52A154KILK2ZGDSY'
CLIENT_SECRET = '2GWT2PANCMXLNAI5MPZLA1WOONKPLST5OARDB3ONOWW4YT0M'
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5LSHG4JHERQ4SHO4D4RFRC50EXWRAXRX52A154KILK2ZGDSY
CLIENT_SECRET:2GWT2PANCMXLNAI5MPZLA1WOONKPLST5OARDB3ONOWW4YT0M


In [36]:
radius=1000
LIMIT=100

In [38]:
latitude = 22.2793278
longitude = 114.1828131
neighborhood_latitude = 22.30383
neighborhood_longitude = 114.18297

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=5LSHG4JHERQ4SHO4D4RFRC50EXWRAXRX52A154KILK2ZGDSY&client_secret=2GWT2PANCMXLNAI5MPZLA1WOONKPLST5OARDB3ONOWW4YT0M&v=20180604&ll=22.30383,114.18297&radius=1000&limit=100'

Send request and examine the results.

In [39]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5de5d7c114a126001ba5f137'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Hong Kong',
  'headerFullLocation': 'Hong Kong',
  'headerLocationGranularity': 'city',
  'totalResults': 109,
  'suggestedBounds': {'ne': {'lat': 22.31283000900001,
    'lng': 114.19267963780591},
   'sw': {'lat': 22.294829990999993, 'lng': 114.17326036219409}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d26701677a2a1cde9bf4fb7',
       'name': 'Hotel ICON (唯港薈)',
       'location': {'address': '17 Science Museum Road, Tsim Sha Tsui East',
        'lat': 22.300801104582376,
        'lng': 114.17971994113918,
        'labeledLatLngs': [{'label': 'display',
        

### Explore districts

In [40]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [41]:
hk_venues = getNearbyVenues(names=df_hk['District'],
                                   latitudes=df_hk['Latitude'],
                                   longitudes=df_hk['Longitude']
                                  )

Sha Tin 
Eastern
Sai Kung 
Yuen Long
Kwun Tong


Check the size of resulting dataframe.

In [42]:
print(hk_venues.shape)
hk_venues.head()

(335, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Sha Tin,22.3771,114.1974,MTR Sha Tin Wai Station (港鐵沙田圍站),22.377042,114.195034,Train Station
1,Sha Tin,22.3771,114.1974,Keung Kee Seafood Restaurant (強記雞粥海鮮酒家),22.381146,114.195523,Seafood Restaurant
2,Sha Tin,22.3771,114.1974,Sha Tin Park (沙田公園),22.378251,114.188151,Park
3,Sha Tin,22.3771,114.1974,叉燒大王,22.381281,114.196036,Chinese Restaurant
4,Sha Tin,22.3771,114.1974,Royal Park Hotel (帝都酒店),22.379678,114.188542,Hotel


In [44]:
hk_venues.groupby('District').count()

,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
District,,,,,,
Eastern,90,90,90,90,90,90
Kwun Tong,87,87,87,87,87,87
Sai Kung,62,62,62,62,62,62
Sha Tin,45,45,45,45,45,45
Yuen Long,51,51,51,51,51,51


In [46]:
print('There are {} uniques categories.'.format(len(hk_venues['Venue Category'].unique())))

There are 103 uniques categories.


As we can see so far, here we are able to find the number of restaurants and the number of category in each district. Later, let's find out more in details. 

### Pre-Processing

In [47]:
# one hot encoding
hk_onehot = pd.get_dummies(hk_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hk_onehot['District'] = hk_venues['District'] 

# move neighborhood column to the first column
fixed_columns = [hk_onehot.columns[-1]] + list(hk_onehot.columns[:-1])
hk_onehot = hk_onehot[fixed_columns]

hk_onehot.head()

,District,American Restaurant,Art Gallery,Asian Restaurant,BBQ Joint,Bakery,Bar,Beach,Beijing Restaurant,Betting Shop,...,Thai Restaurant,Theater,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Veterinarian,Vietnamese Restaurant,Volleyball Court,Wine Shop
0,Sha Tin,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,Sha Tin,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Sha Tin,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Sha Tin,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Sha Tin,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
hk_onehot.shape

(335, 104)

In [50]:
hk_grouped = hk_onehot.groupby('District').mean().reset_index()
hk_grouped

,District,American Restaurant,Art Gallery,Asian Restaurant,BBQ Joint,Bakery,Bar,Beach,Beijing Restaurant,Betting Shop,...,Thai Restaurant,Theater,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Veterinarian,Vietnamese Restaurant,Volleyball Court,Wine Shop
0,Eastern,0.000000,0.011111,0.000000,0.000000,0.022222,0.022222,0.000000,0.000000,0.000000,...,0.022222,0.000000,0.011111,0.000000,0.011111,0.022222,0.000000,0.022222,0.011111,0.000000
1,Kwun Tong,0.000000,0.011494,0.022989,0.011494,0.011494,0.000000,0.000000,0.011494,0.000000,...,0.011494,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011494,0.000000,0.000000
2,Sai Kung,0.016129,0.000000,0.000000,0.032258,0.000000,0.016129,0.016129,0.000000,0.016129,...,0.080645,0.000000,0.000000,0.000000,0.000000,0.000000,0.016129,0.000000,0.000000,0.016129
3,Sha Tin,0.000000,0.000000,0.022222,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.022222,0.000000,0.000000,0.044444,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Yuen Long,0.000000,0.000000,0.000000,0.000000,0.019608,0.000000,0.000000,0.000000,0.000000,...,0.019608,0.019608,0.000000,0.019608,0.000000,0.019608,0.000000,0.000000,0.000000,0.000000


Check the size of grouped dataframe.

In [51]:
hk_grouped.shape

(5, 104)

Let's see what are the top 5 most common venues in each district.

In [54]:
num_top_venues = 5

for hood in hk_grouped['District']:
    print("----"+hood+"----")
    temp = hk_grouped[hk_grouped['District'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Eastern----
                  venue  freq
0          Noodle House  0.08
1                 Hotel  0.07
2    Chinese Restaurant  0.07
3  Cantonese Restaurant  0.06
4           Coffee Shop  0.06


----Kwun Tong----
                  venue  freq
0    Chinese Restaurant  0.09
1  Fast Food Restaurant  0.08
2           Coffee Shop  0.06
3                  Café  0.06
4        Cha Chaan Teng  0.05


----Sai Kung ----
                venue  freq
0  Seafood Restaurant  0.10
1                Café  0.10
2     Thai Restaurant  0.08
3  Chinese Restaurant  0.05
4                 Pub  0.03


----Sha Tin ----
                  venue  freq
0    Chinese Restaurant  0.22
1  Fast Food Restaurant  0.07
2           Pizza Place  0.04
3                  Café  0.04
4  Cantonese Restaurant  0.04


----Yuen Long----
                  venue  freq
0    Chinese Restaurant  0.14
1          Noodle House  0.14
2  Fast Food Restaurant  0.08
3   Japanese Restaurant  0.08
4         Shopping Mall  0.06




In [55]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood.

In [61]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
District_venues_sorted = pd.DataFrame(columns=columns)
District_venues_sorted['District'] = hk_grouped['District']

for ind in np.arange(hk_grouped.shape[0]):
    District_venues_sorted.iloc[ind, 1:] = return_most_common_venues(hk_grouped.iloc[ind, :], num_top_venues)

District_venues_sorted.head()

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Eastern,Noodle House,Hotel,Chinese Restaurant,Cantonese Restaurant,Coffee Shop,Sushi Restaurant,Dessert Shop,Italian Restaurant,Café,Korean Restaurant
1,Kwun Tong,Chinese Restaurant,Fast Food Restaurant,Coffee Shop,Café,Cha Chaan Teng,Noodle House,Hong Kong Restaurant,Japanese Restaurant,Sushi Restaurant,Restaurant
2,Sai Kung,Café,Seafood Restaurant,Thai Restaurant,Chinese Restaurant,Pub,Dessert Shop,Coffee Shop,Pizza Place,Burger Joint,Park
3,Sha Tin,Chinese Restaurant,Fast Food Restaurant,Cantonese Restaurant,Train Station,Hotel,Café,Convenience Store,Pizza Place,Playground,Pool
4,Yuen Long,Chinese Restaurant,Noodle House,Japanese Restaurant,Fast Food Restaurant,Shopping Mall,Dessert Shop,Café,Coffee Shop,Pool,Indian Restaurant


Let's see if there is any Jewish restaurant in these district as we would like to check whether we will have competitor.

# Clustering

Run k-means to cluster the neighborhood into 5 clusters.

In [66]:
# set number of clusters
kclusters = 5

hk_grouped_clustering = hk_grouped.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hk_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 2, 1, 4, 0], dtype=int32)

Create a new dataframe that includes the cluster for each neighborhood.

In [67]:
# add clustering labels
District_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

hk_merged = df_hk

# merge hk_grouped with hk_data to add latitude/longitude for each neighborhood
hk_merged = hk_merged.join(District_venues_sorted.set_index('District'), on='District')

hk_merged.head() # check the last columns!

,District,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Station
0,Sha Tin,22.377100,114.197400,4,Chinese Restaurant,Fast Food Restaurant,Cantonese Restaurant,Train Station,Hotel,Café,Convenience Store,Pizza Place,Playground,Pool,Yes
1,Eastern,22.288500,114.192800,3,Noodle House,Hotel,Chinese Restaurant,Cantonese Restaurant,Coffee Shop,Sushi Restaurant,Dessert Shop,Italian Restaurant,Café,Korean Restaurant,Yes
2,Sai Kung,22.381400,114.273330,1,Café,Seafood Restaurant,Thai Restaurant,Chinese Restaurant,Pub,Dessert Shop,Coffee Shop,Pizza Place,Burger Joint,Park,Yes
3,Yuen Long,22.444500,114.022200,0,Chinese Restaurant,Noodle House,Japanese Restaurant,Fast Food Restaurant,Shopping Mall,Dessert Shop,Café,Coffee Shop,Pool,Indian Restaurant,Yes
4,Kwun Tong,22.311774,114.226209,2,Chinese Restaurant,Fast Food Restaurant,Coffee Shop,Café,Cha Chaan Teng,Noodle House,Hong Kong Restaurant,Japanese Restaurant,Sushi Restaurant,Restaurant,Yes


### Visualize the result

In [69]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(hk_merged['Latitude'], hk_merged['Longitude'], hk_merged['District'], hk_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster+1), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 1

In [70]:
hk_merged.loc[hk_merged['Cluster Labels'] == 0, hk_merged.columns[[1] + list(range(5, hk_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Station
3,22.4445,Noodle House,Japanese Restaurant,Fast Food Restaurant,Shopping Mall,Dessert Shop,Café,Coffee Shop,Pool,Indian Restaurant,Yes


#### Cluster 2

In [71]:
hk_merged.loc[hk_merged['Cluster Labels'] == 1, hk_merged.columns[[1] + list(range(5, hk_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Station
2,22.3814,Seafood Restaurant,Thai Restaurant,Chinese Restaurant,Pub,Dessert Shop,Coffee Shop,Pizza Place,Burger Joint,Park,Yes


#### Cluster 3

In [72]:
hk_merged.loc[hk_merged['Cluster Labels'] == 2, hk_merged.columns[[1] + list(range(5, hk_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Station
4,22.311774,Fast Food Restaurant,Coffee Shop,Café,Cha Chaan Teng,Noodle House,Hong Kong Restaurant,Japanese Restaurant,Sushi Restaurant,Restaurant,Yes


#### Cluster 4

In [73]:
hk_merged.loc[hk_merged['Cluster Labels'] == 3, hk_merged.columns[[1] + list(range(5, hk_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Station
1,22.2885,Hotel,Chinese Restaurant,Cantonese Restaurant,Coffee Shop,Sushi Restaurant,Dessert Shop,Italian Restaurant,Café,Korean Restaurant,Yes


#### Cluster 5

In [74]:
hk_merged.loc[hk_merged['Cluster Labels'] == 4, hk_merged.columns[[1] + list(range(5, hk_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Station
0,22.3771,Fast Food Restaurant,Cantonese Restaurant,Train Station,Hotel,Café,Convenience Store,Pizza Place,Playground,Pool,Yes


# Results

Based on our analysis using scoring system, the types of restaurant nearby in the same district(potential competitor), and clustering, we found that Sha Tin has the overall best conditions for our new flagship restaurant in Hong Kong. 

We first narrowed down the scope by using scoring system to select the 5 districts with the top final score. Given that among the 18 districts, people who live in Sha tin have above-average monthly income, good amount of young to middle-aged group, also with reasonable shop rent. Moreover, within the district, there is no Jewish restaurant that makes no direct competition to our client. With the suitable target customer and also convenient transportation, we can suggest that Sha Tin could be a good place for the new restaurant. 

# Conclusion

The purpose of this project was to identify which area in Hong Kong will be the best choice for our client to open their first Jewish restaurant. Since they already set the criteria of target customers based on income and age range, and taking the shop rent into consideration, we have narrowed down the district based on the scoring system as the first step. 

In order to aid our client to look for the optimal location for a new Jewish restaurant, we categorized the restaurant type from Foursquare data so we can identify whether there is any direct competitor in the same district. Clustering helped us to see the density of the type of the restaurant to come up with the final conclusion. 


Final decission on optimal location of the Jewish restaurant location will be made by stakeholders based on specific characteristics of district and locations in every recommended zone, taking into consideration additional factors like convenience of each location (proximity to station), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every district etc.

